In [79]:
import pandas as pd
from icecream import ic

In [80]:
df = pd.read_csv('data/data_57.csv', index_col=0)
df = df[~df.text_input.isnull()]
df['true_cluster'] = df.cluster.copy()
df.head()

,id,text_input,cluster,true_cluster
5,90663250,?,NaN,NaN
6,90683982,?,NaN,NaN
7,90669451,?,NaN,NaN
8,90662341,?,NaN,NaN
9,90661642,?,NaN,NaN


In [120]:
known_labels = {c for c in df.cluster.unique() if c is not np.nan}
known_labels

{'BUX',
 'Bitpanda',
 'Comdirect',
 'Consorsbank',
 'DKB',
 'DeGiro',
 'Finanzen.net Zero',
 'Flatex',
 'ING',
 'Justtrade',
 'Nextmarkets',
 'Onvista',
 'Robinhood',
 'S-Broker',
 'Scalable Capital',
 'Smartbroker',
 'Trade Republic',
 'Trading 212',
 'Vivid Money',
 'eToro'}

# Propblems

* Fuzzy match false positive: brokers -> S-broker. We need to use the information that "brokers" is a common word, while S-broker is something unique?

* 'DKB' vs 'dks' - why we need ratio instead of absolute distance
* 'Vivid' vs 'Vivid Money' - both words are common (more or less), but in this context 'vivid' is specific
* 'plus500' vs '500plus' - fuzzy diff will be huge



# Preprocessing

In [115]:
import unicodedata, string
import unidecode, re
from unidecode import unidecode

def rem_punctuation(x):
    return x.translate(str.maketrans('', '', string.punctuation))

def normalize(s):
#     return unicodedata.normalize('NFKD', s)
    return unidecode(s)

def rem_extra_spaces(s):
    s = re.sub('  +', ' ', s)
    s = re.sub('\n', ' ', s)
    s = re.sub('\t', ' ', s)
    return s

    
def preprocess(x):
    if not x:
        return None
    
    x = rem_punctuation(x)
    x = rem_extra_spaces(x)
    x = normalize(x)
    x = x.lower()
    x = x.strip()
    return x or None

preprocess('Dafür muss man sich auskennen',)



'dafur muss man sich auskennen'

In [117]:
df['text_pro'] = df.text_input.apply(preprocess)
df = df[~df.text_pro.isnull()]
df

,id,text_input,cluster,true_cluster,found_clusters,text_pro
17,90669113,0,NaN,NaN,None,0
18,90665568,0,NaN,NaN,None,0
19,90671351,"0,000",NaN,NaN,None,0000
20,90670294,2,NaN,NaN,None,2
21,90679455,3,NaN,NaN,None,3
...,...,...,...,...,...,...
1592,90666125,Weiß nicht,NaN,NaN,None,weiss nicht
1593,90682704,xtb,NaN,NaN,None,xtb
1594,90676142,zero,Finanzen.net Zero,Finanzen.net Zero,None,zero
1595,90684425,Zfugiv,NaN,NaN,None,zfugiv


In [121]:
known_labels = {
    preprocess(l):l
    for l in known_labels
}
known_labels

{'trading 212': 'Trading 212',
 'onvista': 'Onvista',
 'nextmarkets': 'Nextmarkets',
 'consorsbank': 'Consorsbank',
 'bux': 'BUX',
 'finanzennet zero': 'Finanzen.net Zero',
 'ing': 'ING',
 'degiro': 'DeGiro',
 'sbroker': 'S-Broker',
 'dkb': 'DKB',
 'vivid money': 'Vivid Money',
 'trade republic': 'Trade Republic',
 'smartbroker': 'Smartbroker',
 'justtrade': 'Justtrade',
 'scalable capital': 'Scalable Capital',
 'etoro': 'eToro',
 'bitpanda': 'Bitpanda',
 'comdirect': 'Comdirect',
 'robinhood': 'Robinhood',
 'flatex': 'Flatex'}

# Fuzzy search - classification

In [135]:
extract('500plus', ['plus500',])

[('plus500', 57.14285714285714, 0)]

In [132]:

from rapidfuzz import fuzz
from rapidfuzz.process import extractOne, extract
# from tabulate import tabulate
# from textwrap import fill

def ffind(text, targets):
    r = extractOne(text, targets, 
                   scorer=fuzz.QRatio, 
                   score_cutoff=70)
    r2 = extractOne(text, targets, 
               scorer=fuzz.token_ratio, 
               score_cutoff=89)

    return r or r2


def ffind_get_label(text, targets):
    res = ffind(text, targets)
    if res:
        return res[0]
ffind_get_label('dkb', targets=known_labels.keys())

'dkb'

In [128]:
df['found_clusters'] = df.text_pro.apply(
     lambda t: ffind_get_label(t, targets=known_labels)
)
df.found_clusters.dropna()

/var/folders/6r/ymffrb5j5m5fjp9sdqd_jmyr0000gn/T/ipykernel_33031/230979205.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['found_clusters'] = df.text_pro.apply(


99            Bitpanda
100           Bitpanda
101           Bitpanda
102           Bitpanda
103           Bitpanda
             ...      
1581    Trade Republic
1582    Trade Republic
1583       Trading 212
1584       Trading 212
1585    Trade Republic
Name: found_clusters, Length: 445, dtype: object

In [129]:
confused_df = df[~df.true_cluster.isna()][
    df.found_clusters != df.true_cluster
]
confused_df.found_clusters.value_counts(dropna=False)

/var/folders/6r/ymffrb5j5m5fjp9sdqd_jmyr0000gn/T/ipykernel_33031/1731788960.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  confused_df = df[~df.true_cluster.isna()][


found_clusters
None    202
Name: count, dtype: int64

In [130]:
confused_df[confused_df.found_clusters.isna()]

,id,text_input,cluster,true_cluster,found_clusters,text_pro
128,90684999,brokers,S-Broker,S-Broker,None,brokers
133,90670739,capital,Scalable Capital,Scalable Capital,None,capital
134,90676823,Capital,Scalable Capital,Scalable Capital,None,capital
202,90681005,consors,Consorsbank,Consorsbank,None,consors
203,90675156,consors,Consorsbank,Consorsbank,None,consors
...,...,...,...,...,...,...
1540,90668451,Scalable,Scalable Capital,Scalable Capital,None,scalable
1541,90662959,Scalable,Scalable Capital,Scalable Capital,None,scalable
1551,90678609,Scapable,Scalable Capital,Scalable Capital,None,scapable
1589,90675476,Vivid,Vivid Money,Vivid Money,None,vivid


In [43]:
df.text_input.to_list()

['?',
 '?',
 '?',
 '?',
 '?',
 '???????',
 '/',
 '.',
 '...',
 '.....',
 '......',
 '..........',
 '0',
 '0',
 '0,000',
 '2',
 '3',
 '4',
 '500plus',
 '56',
 '7865r',
 '8989',
 'aaaaaaaaaaaaaaaaaaaaaa',
 'Aktien',
 'Aktien 24',
 'Aktien Aktuell',
 'Alisnz',
 'All Börse',
 'Allvest',
 'amazon',
 'amazon',
 'Amazon',
 'Amazon',
 'Amazon',
 'Amazon',
 'an eine',
 'an keine',
 'an keine bestimmte zuerst',
 'an nichts',
 'App',
 'Apple',
 'Apple',
 'Ativi',
 'auch',
 'AXA',
 'bank',
 'bank',
 'Bank',
 'Bank',
 'Bank',
 'Bank',
 'Bank',
 'Bank',
 'Bank depot',
 'banken',
 'Banken',
 'Banken',
 'Banken',
 'Bankinstitut',
 'Bet',
 'Bet',
 'Bevestor',
 'Bfnfkd',
 'binance',
 'binance',
 'binance',
 'Binance',
 'Binance',
 'Binance',
 'Binance',
 'Binance',
 'Binance',
 'Binance',
 'Binance',
 'Binance',
 'Binance',
 'Bing X',
 'Bison',
 'Bit',
 'Bitcion',
 'bitcoin',
 'bitcoin',
 'Bitcoin',
 'Bitcoin',
 'Bitcoin',
 'Bitcoin',
 'Bitcoin',
 'Bitcoin',
 'Bitcoin',
 'Bitcoin',
 'Bitcoin',
 'Bitcoin

In [27]:
df.cluster.value_counts(dropna=False)

cluster
NaN                  951
eToro                133
Trade Republic       109
ING                   73
Comdirect             64
Scalable Capital      57
Consorsbank           39
Flatex                39
Smartbroker           24
DKB                   20
S-Broker              19
Finanzen.net Zero     18
Bitpanda              14
Onvista               11
Robinhood              8
Trading 212            5
DeGiro                 4
Vivid Money            4
Justtrade              3
BUX                    2
Nextmarkets            1
Name: count, dtype: int64

In [23]:
import json
json.dumps(df.text_input[df.text_input.notna()].tolist())

'["?", "?", "?", "?", "?", "???????", "/", ".", "...", ".....", "......", "..........", "0", "0", "0,000", "2", "3", "4", "500plus", "56", "7865r", "8989", "aaaaaaaaaaaaaaaaaaaaaa", "Aktien", "Aktien 24", "Aktien Aktuell", "Alisnz", "All B\\u00f6rse", "Allvest", "amazon", "amazon", "Amazon", "Amazon", "Amazon", "Amazon", "an eine", "an keine", "an keine bestimmte zuerst", "an nichts", "App", "Apple", "Apple", "Ativi", "auch", "AXA", "bank", "bank", "Bank", "Bank", "Bank", "Bank", "Bank", "Bank", "Bank depot", "banken", "Banken", "Banken", "Banken", "Bankinstitut", "Bet", "Bet", "Bevestor", "Bfnfkd", "binance", "binance", "binance", "Binance", "Binance", "Binance", "Binance", "Binance", "Binance", "Binance", "Binance", "Binance", "Binance", "Bing X", "Bison", "Bit", "Bitcion", "bitcoin", "bitcoin", "Bitcoin", "Bitcoin", "Bitcoin", "Bitcoin", "Bitcoin", "Bitcoin", "Bitcoin", "Bitcoin", "Bitcoin", "Bitcoin.com", "Bitcoins", "Bitcoins", "bitpanda", "Bitpanda", "Bitpanda", "Bitpanda", "Bitp

In [12]:
df.text_input.value_counts(dropna=False)

text_input
keine              68
Keine              44
etoro              42
Trade Republic     36
Etoro              35
                   ..
Keine Angabe        1
Kein bestimmtes     1
Kein Ahnung         1
keien               1
Zzz                 1
Name: count, Length: 632, dtype: int64

In [13]:
df.cluster.value_counts(dropna=False)

cluster
NaN                  951
eToro                133
Trade Republic       109
ING                   73
Comdirect             64
Scalable Capital      57
Consorsbank           39
Flatex                39
Smartbroker           24
DKB                   20
S-Broker              19
Finanzen.net Zero     18
Bitpanda              14
Onvista               11
Robinhood              8
Trading 212            5
DeGiro                 4
Vivid Money            4
Justtrade              3
BUX                    2
Nextmarkets            1
Name: count, dtype: int64

In [17]:
df[~df.cluster.notna()].to_csv('data/data_57_not_clustered.csv')